In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range [0,1].
# We transform them to Tensors of normalized range [-1,1]

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#torch._utils.dataloader = torch.utils.DataLoader(dataset, batch_size=1, suffle=False)
test_loader = DataLoader(test_dataset, batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class CovNet(nn.Module):
    def __init__(self):
        super(CovNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        # kernel size of 2 and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5,120) # --> 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CovNet().to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape [4,3,32,32] = 4,3,1024 --> 4 for the batch size, 3 for the channels, 32x32 is image size in pixels
        images = images.to(device)
        labels = labels.to(device)
        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        #Backward and optimize

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training!')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i  in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
              n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/4], Step [2000/12500], Loss: 0.9576
Epoch [1/4], Step [4000/12500], Loss: 1.5396
Epoch [1/4], Step [6000/12500], Loss: 1.7647
Epoch [1/4], Step [8000/12500], Loss: 1.2667
Epoch [1/4], Step [10000/12500], Loss: 1.1052
Epoch [1/4], Step [12000/12500], Loss: 1.1515
Epoch [2/4], Step [2000/12500], Loss: 1.0858
Epoch [2/4], Step [4000/12500], Loss: 1.1132
Epoch [2/4], Step [6000/12500], Loss: 1.3277
Epoch [2/4], Step [8000/12500], Loss: 0.6462
Epoch [2/4], Step [10000/12500], Loss: 2.2661
Epoch [2/4], Step [12000/12500], Loss: 1.4059
Epoch [3/4], Step [2000/12500], Loss: 0.9220
Epoch [3/4], Step [4000/12500], Loss: 0.3422
Epoch [3/4], Step [6000/12500], Loss: 0.2856
Epoch [3/4], Step [8000/12500], Loss: 1.3600
Epoch [3/4], Step [10000/12500], Loss: 1.9096
Epoch [3/4], Step [12000/12500], Loss: 0.8000
Epoch [4/4], Step [2000/12500], Loss: 1.1050
Epoch [4/4], Step [4000/12500], Loss: 0.9611
Epoch [4/4], Step 